<a href="https://colab.research.google.com/github/AI4Finance-Foundation/FinRL/blob/master/tutorials/3-Practical/FinRL_PaperTrading_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import sys, os

root_path = os.getcwd()
root_path = os.path.join(root_path, '..', '..')
print(os.path.abspath(root_path))
sys.path.append(root_path)
root_path = os.path.join(root_path, '..', 'ElegantRL')
print(os.path.abspath(root_path))
sys.path.append(root_path)

/home/quantumiracle/research/FinRL
/home/quantumiracle/research/ElegantRL


In [7]:
from finrl.train import train, load_df
from finrl.test import test
from finrl.config_tickers import DOW_30_TICKER
from finrl.config import INDICATORS
from finrl.meta.env_stock_trading.env_stocktrading_np import StockTradingEnv
from finrl.plot import backtest_stats, get_daily_return, get_baseline # backtest_plot
from common import *
import datetime as dt
from finrl.plot import *
import pandas as pd

In [8]:
ticker_list = DOW_30_TICKER
action_dim = len(DOW_30_TICKER)
candle_time_interval = '5Min'  # '1Min'

env = StockTradingEnv

ERL_PARAMS = {"learning_rate": 3e-6, "batch_size": 2048, "gamma": 0.985,
              "seed": 312, "net_dimension": 512, "target_step": 5000, "eval_gap": 30,
              "eval_times": 1}

train_start_date = '2022-6-11'
train_end_date = '2022-8-11'
test_start_date = '2021-8-11'
test_end_date = '2022-9-1'
baseline_ticker = 'AXP'

model_name = 'ppo'
MODEL_IDX = f'{model_name}_{train_start_date}_{train_end_date}'


In [9]:
def backtest_plot(
        account_value,
        baseline_df,
        value_col_name="account_value",
):
    df = deepcopy(account_value)
    # print('date', len(df))
    # print(type(df["date"][0]), df["date"][0])
    df["date"] = pd.to_datetime(df["date"])
    # df["date"] = pd.Timestamp(df["date"]).tz_localize("America/New_York")
    # df["date"] = df["date"].tz_localize("America/New_York")
    test_returns = get_daily_return(df, value_col_name=value_col_name)

    baseline_df["date"] = pd.to_datetime(baseline_df["date"], format="%Y-%m-%d")
    baseline_df = pd.merge(df[["date"]], baseline_df, how="left", on="date")
    baseline_df = baseline_df.fillna(method="ffill").fillna(method="bfill")
    baseline_returns = get_daily_return(baseline_df, value_col_name="close")

    with pyfolio.plotting.plotting_context(font_scale=1.1):
        pyfolio.create_full_tear_sheet(
            returns=test_returns, benchmark_rets=baseline_returns, set_context=False
        )



In [10]:
# train(start_date=train_start_date,
#       end_date=train_end_date,
#       ticker_list=ticker_list,
#       data_source='alpaca',
#       time_interval=candle_time_interval,
#       technical_indicator_list=INDICATORS,
#       drl_lib='elegantrl',
#       #       drl_lib='rllib',
#       #       drl_lib='stable_baselines3',
#       env=env,
#       model_name=model_name,
#       API_KEY=API_KEY,
#       API_SECRET=API_SECRET,
#       API_BASE_URL=API_BASE_URL,
#       erl_params=ERL_PARAMS,
#       cwd=f'./papertrading_erl/{MODEL_IDX}',  # current_working_dir
#       break_step=1e7)

account_value = test(start_date=test_start_date,
                     end_date=test_end_date,
                     ticker_list=ticker_list,
                     data_source='alpaca',
                     time_interval=candle_time_interval,
                     technical_indicator_list=INDICATORS,
                     drl_lib='elegantrl',
                     env=env,
                     model_name='ppo',
                     API_KEY=API_KEY,
                     API_SECRET=API_SECRET,
                     API_BASE_URL=API_BASE_URL,
                     #       erl_params=ERL_PARAMS,
                     cwd=f'./papertrading_erl/{MODEL_IDX}',  # current_working_dir
                     if_plot=True,  # to return a dataframe for backtest_plot
                     break_step=1e7)
print("============== account_value ===========")
print(account_value)



Alpaca successfully connected
Failed to load data from /home/quantumiracle/research/FinRL/finrl/../data/alpaca_2022-6-11_2022-9-1.pkl


UnboundLocalError: local variable 'data' referenced before assignment

In [6]:

# baseline stats
print("==============Get Baseline Stats===========")
# baseline_df_dji = get_baseline(
#     ticker="^DJI",
#     start=test_start_date,
#     end=test_end_date)

baseline_df = load_df(test_start_date, test_end_date)
baseline_df = baseline_df[baseline_df['tic'] == baseline_ticker]
baseline_df = baseline_df.loc[baseline_df['timestamp'].dt.time == dt.time(15, 59)]
baseline_df['date'] = baseline_df['timestamp'].dt.date
stats = backtest_stats(baseline_df, value_col_name='close')
print(stats)



==============Get Baseline Stats===========
Annual return         -0.689856
Cumulative returns    -0.067313
Annual volatility      0.246009
Sharpe ratio          -4.971252
Calmar ratio          -8.277273
Stability              0.837271
Max drawdown          -0.083343
Omega ratio            0.445379
Sortino ratio         -5.649901
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.641447
Daily value at risk   -0.035847
dtype: float64
Annual return         -0.689856
Cumulative returns    -0.067313
Annual volatility      0.246009
Sharpe ratio          -4.971252
Calmar ratio          -8.277273
Stability              0.837271
Max drawdown          -0.083343
Omega ratio            0.445379
Sortino ratio         -5.649901
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.641447
Daily value at risk   -0.035847
dtype: float64


/home/quantumiracle/anaconda3/envs/x/lib/python3.7/site-packages/pyfolio/timeseries.py:724: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  stats = pd.Series()


In [7]:
print("==============Compare to Baseline===========")
backtest_plot(account_value, baseline_df)

==============Compare to Baseline===========


/home/quantumiracle/anaconda3/envs/x/lib/python3.7/site-packages/seaborn/rcmod.py:400: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(mpl.__version__) >= "3.0":
/home/quantumiracle/anaconda3/envs/x/lib/python3.7/site-packages/setuptools/_distutils/version.py:346: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
/home/quantumiracle/anaconda3/envs/x/lib/python3.7/site-packages/pyfolio/timeseries.py:724: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  stats = pd.Series()


Start date,2022-08-11
End date,2022-08-31
Total months,0
,Backtest
Annual return,-69.546%
Cumulative returns,-6.832%
Annual volatility,21.72%
Sharpe ratio,-5.75
Calmar ratio,-7.63
Stability,0.83
Max drawdown,-9.119%


Worst drawdown periods,Net drawdown in %,Peak date,Valley date,Recovery date,Duration
0,9.12,2022-08-18,2022-08-31,NaT,NaN
1,NaN,NaT,NaT,NaT,NaN
2,NaN,NaT,NaT,NaT,NaN
3,NaN,NaT,NaT,NaT,NaN
4,NaN,NaT,NaT,NaT,NaN


/home/quantumiracle/anaconda3/envs/x/lib/python3.7/site-packages/pandas/plotting/_matplotlib/compat.py:13: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  op(LooseVersion(mpl.__version__), LooseVersion(version))
/home/quantumiracle/anaconda3/envs/x/lib/python3.7/site-packages/pandas/plotting/_matplotlib/tools.py:321: MatplotlibDeprecationWarning: 
The is_last_row function was deprecated in Matplotlib 3.4 and will be removed two minor releases later. Use ax.get_subplotspec().is_last_row() instead.
  if ax.is_last_row():
/home/quantumiracle/anaconda3/envs/x/lib/python3.7/site-packages/pandas/plotting/_matplotlib/tools.py:331: MatplotlibDeprecationWarning: 
The is_first_col function was deprecated in Matplotlib 3.4 and will be removed two minor releases later. Use ax.get_subplotspec().is_first_col() instead.
  if ax.is_first_col():
/home/quantumiracle/anaconda3/envs/x/lib/python3.7/site-packages/pyfolio/plotting.py:806: DeprecationWarning: Th